In [ ]:
import math

# Forward Kinematics
def fk_3dof(theta1_deg, theta2_deg, theta3_deg):
    # 링크 길이 (cm)
    L1 = 7.8
    L2 = 8.3
    L3 = 3.9

    # joint_limits = {
    # 'theta1': (-30, 210),     # base 회전
    # 'theta2': (-30, 210),    # shoulder
    # 'theta3': (-30, 210),     # elbow
    # }

    # # 범위 검사
    # input_angles = {'theta1': theta1_deg, 'theta2': theta2_deg, 'theta3': theta3_deg}
    # for name, angle in input_angles.items():
    #     min_angle, max_angle = joint_limits[name]
    #     if not (min_angle <= angle <= max_angle):
    #         return None, f"[오류] {name}={angle:.1f}°: 가동 범위 초과"

    # === 각보정 ===
    theta1_deg = theta1_deg - 90
    theta3_deg = 90 - theta3_deg 

        # 각도 변환
    t1 = math.radians(theta1_deg)
    t2 = math.radians(theta2_deg)
    t3 = math.radians(theta3_deg)

    # 위치 계산
    z = L1 + L2 * math.sin(t2) + L3 * math.sin(t2+t3)
    r = L2 * math.cos(t2) + L3 * math.cos(t2+t3)
    x = r * math.cos(t1)
    y = r * math.sin(t1)


    return (x, y, z)
# , None

def ik_3dof(x, y, z):
    L1 = 7.8
    L2 = 8.3
    L3 = 3.9
    # theta1
    theta1 = math.atan2(y, x)
    theta1_deg = math.degrees(theta1)
    theta1_deg = round(theta1 + 90, 2) # 각보정

    # 조인트2 기준으로 평면 거리(r) 및 높이(z_offset)
    r = math.hypot(x, y)
    z_offset = z - L1  # shoulder (조인트2) 기준으로 변환
    d = math.hypot(r, z_offset)

    if d > (L2 + L3):
        return None, "Unreachable: 목표 좌표가 팔 길이보다 멉니다."

    # cos_theta3 = (L2**2 + L3**2 - r**2 - z_offset**2) / (2 * L2 * L3) # L2, L3 사이 끼인각
    cos_theta3 = (-L2**2 - L3**2 + r**2 + z_offset**2) / (2 * L2 * L3) # 실제 theta2 각 (180 - 끼인각 = theta2)
    
    # cos_theta3 = max(-1.0, min(1.0, cos_theta3))  # 안전 범위로 클램핑

    # if not -1.0 <= cos_theta3 <= 1.0:
    #     raise ValueError("Target position is outside the reachable workspace.")
    
    # # theta3
    # theta3 = math.atan2(math.sqrt(1-(cos_theta3)**2),cos_theta3)
    # theta3_deg = math.degrees(theta3)
    # theta3_deg = round(90 - theta3_deg, 2) # 각보정

    if not -1.0 <= cos_theta3 <= 1.0:
        theta3_deg = None
        cos_theta3 = max(-1.0, min(1.0, cos_theta3))
        theta3 = math.acos(cos_theta3)
    else:
        # 안전 클램핑 후 계산
        cos_theta3 = max(-1.0, min(1.0, cos_theta3))
        theta3 = math.acos(cos_theta3)
        theta3_deg = round(90 - math.degrees(theta3), 2)
    


    # theta2
    alpha = math.atan2(z_offset, r)
    beta = math.acos((L2**2 + d**2 - L3**2) / 2*L2*d)
    # beta = math.atan2(L3 * math.sin(theta3), L2 + (L3 * math.cos(theta3)))
    theta2 = alpha + beta
    theta2_deg = round(math.degrees(theta2), 2)

    return (theta1_deg, theta2_deg, theta3_deg)

In [198]:
# FK → IK → FK round-trip 확인
import pandas as pd
a_list = []
re_a_list = []
for x in range(0,190, 10):
    for y in range(0,190, 10):
        for z in range(0,190, 10):
            angles = (x, y, z)
            xyz = fk_3dof(*angles)
            recovered_angles = ik_3dof(*xyz)
            a_list.append(angles)
            re_a_list.append(recovered_angles)
angle_list_concat = pd.DataFrame([
    (
        a[0], a[1], a[2],
        r[0] if r is not None and len(r) > 0 else None,
        r[1] if r is not None and len(r) > 1 else None,
        r[2] if r is not None and len(r) > 2 else None
    )
    for a, r in zip(a_list, re_a_list)
], columns=['x', 'y', 'z', 'rx', 'ry', 'rz'])
angle_list_concat

,x,y,z,rx,ry,rz
0,0,0,0,-91.57,50.34,0.0
1,0,0,10,-91.57,46.33,10.0
2,0,0,20,-91.57,41.65,20.0
3,0,0,30,-91.57,36.48,30.0
4,0,0,40,-91.57,30.91,40.0
...,...,...,...,...,...,...
6854,180,180,140,-91.57,30.91,40.0
6855,180,180,150,-91.57,36.48,30.0
6856,180,180,160,-91.57,41.65,20.0
6857,180,180,170,-91.57,46.33,10.0


In [184]:
angle_list_concat.to_csv("ik_fk_roundtrip.csv", index=False)

In [10]:
# FK, angle -> coordinate
result = fk_3dof(90,90,90)
# result = result[0]
print(result)

(7.470345474798854e-16, 0.0, 20.0)


In [11]:
ik_3dof(result[0],result[1],result[2])

ValueError: math domain error

In [201]:
import pandas as pd
ik_data = []
fk_data = []
for i in range(0, 190, 10):
    result = fk_3dof(i,90,180)
    # result = result[0]
    ik_data.append(ik_3dof(result[0],result[1],result[2]))
    fk_data.append(result)


ik_data_df = pd.DataFrame(ik_data, columns=["theta1", "theta2", "theta3"])
fk_data_df = pd.DataFrame(fk_data, columns=["x", "y", "z"])
data_concat = pd.concat([ik_data_df, fk_data_df], axis=1)
print(data_concat)
# print(fk_data_concat[['x','y','z']])

    theta1  theta2  theta3             x         y     z
0   -91.57    90.0     0.0  2.388061e-16 -3.900000  16.1
1   -91.40    90.0     0.0  6.772279e-01 -3.840750  16.1
2   -91.22    90.0     0.0  1.333879e+00 -3.664801  16.1
3   -91.05    90.0     0.0  1.950000e+00 -3.377499  16.1
4   -90.87    90.0     0.0  2.506872e+00 -2.987573  16.1
5   -90.70    90.0     0.0  2.987573e+00 -2.506872  16.1
6   -90.52    90.0     0.0  3.377499e+00 -1.950000  16.1
7   -90.35    90.0     0.0  3.664801e+00 -1.333879  16.1
8   -90.17    90.0     0.0  3.840750e+00 -0.677228  16.1
9   -90.00    90.0     0.0  3.900000e+00  0.000000  16.1
10  -89.83    90.0     0.0  3.840750e+00  0.677228  16.1
11  -89.65    90.0     0.0  3.664801e+00  1.333879  16.1
12  -89.48    90.0     0.0  3.377499e+00  1.950000  16.1
13  -89.30    90.0     0.0  2.987573e+00  2.506872  16.1
14  -89.13    90.0     0.0  2.506872e+00  2.987573  16.1
15  -88.95    90.0     0.0  1.950000e+00  3.377499  16.1
16  -88.78    90.0     0.0  1.3

In [120]:
xy_list = data_concat[['x', 'y']]
xy_list

,x,y
0,4.574267e-32,-7.470345e-16
1,1.297212e-16,-7.356854e-16
2,2.555009e-16,-7.019829e-16
3,3.735173e-16,-6.469509e-16
4,4.801846e-16,-5.722617e-16
5,5.722617e-16,-4.801846e-16
6,6.469509e-16,-3.735173e-16
7,7.019829e-16,-2.555009e-16
8,7.356854e-16,-1.297212e-16
9,7.470345e-16,0.000000e+00


In [123]:
xy_list = data_concat[['x', 'y']]
theta1_list = []
for i, val in xy_list.iterrows():
    x = val[0]
    y = val[1]
    theta1 = math.atan2(y, x)
    theta1_deg = math.degrees(theta1)
    theta1_deg = round(theta1 + 90, 2) # 각보정
    theta1_list.append(theta1_deg)

print(theta1_list)



[88.43, 88.6, 88.78, 88.95, 89.13, 89.3, 89.48, 89.65, 89.83, 90.0, 90.17, 90.35, 90.52, 90.7, 90.87, 91.05, 91.22, 91.4, 91.57]


/tmp/ipykernel_4909/3539695030.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = val[0]
/tmp/ipykernel_4909/3539695030.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = val[1]


In [6]:
import random
import math

# 로봇팔 링크 길이
L1 = 7.8
L2 = 8.3
L3 = 3.9
MAX_REACH = L2 + L3

def generate_valid_xyz():
    while True:
        # z 범위 내 임의 생성
        z = random.uniform(L1, L1 + MAX_REACH)
        # r은 x, y 평면상의 거리
        r = random.uniform(0, MAX_REACH)

        # 각도 랜덤 (theta1 ∈ [0, 180]) → 반원 내 좌표 생성
        angle_rad = math.radians(random.uniform(0, 180))
        x = r * math.sin(angle_rad)
        y = r * math.cos(angle_rad)

        # 조인트2 기준 거리 d 계산
        z_offset = z - L1
        d = math.hypot(r, z_offset)

        if d <= MAX_REACH:
            return round(x, 2), round(y, 2), round(z, 2)


In [66]:
rand_coord = generate_valid_xyz()
rand_coord

(4.11, 0.13, 10.42)

In [25]:
import matplotlib.pyplot as plt

# x = rand_coord[0]
# y = rand_coord[1]
# z = rand_coord[2]

x = 3
y = 0
z = 15

L1 = 7.8
L2 = 8.3
L3 = 3.9

results = []
for x in range(0, 11):
    for z in range(0, 15):
        r = math.hypot(x, y)
        z_offset = z - L1  # shoulder (조인트2) 기준으로 변환
        d = math.hypot(r, z_offset)
        try : 
            cos_theta3 = (-L2**2 - L3**2 + r**2 + z_offset**2) / (2 * L2 * L3) # 실제 theta2 각 (180 - 끼인각 = theta2)
            if cos_theta3 < -1 or cos_theta3 >1:
                raise ValueError("cosθ3 out of domain")
            theta3 = math.atan2(math.sqrt(1-(cos_theta3)**2),cos_theta3)
            results.append((x, z, cos_theta3, math.degrees(theta3)))
        except:
            results.append((x, z, None, None))

results = pd.DataFrame(results, columns=["x", "z", "cos_theta3", "theta3"])
results

,x,z,cos_theta3,theta3
0,0,0,-0.359283,111.056187
1,0,1,-0.584801,125.788915
2,0,2,-0.779425,141.207995
3,0,3,-0.943157,160.588720
4,0,4,NaN,NaN
...,...,...,...,...
160,10,10,0.320358,71.315402
161,10,11,0.403769,66.185996
162,10,12,0.518072,58.796967
163,10,13,0.663268,48.450377
